# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [2]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [3]:
profile = profile[(profile['age']<100) & (profile['gender']!=None)]

In [4]:
profile.head()

,age,became_member_on,gender,id,income
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
5,68,20180426,M,e2127556f4f64592b11af22de27a7932,70000.0
8,65,20180209,M,389bc3fa690240e798340f5a15918d5c,53000.0
12,58,20171111,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0


In [5]:
len(profile)

14808

In [6]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [7]:
len(transcript)

306534

In [8]:
combo = pd.merge(profile, transcript, how='inner', left_on=['id'], right_on=['person']).rename(columns={'id': 'person_id'})

In [9]:
len(combo)

272430

In [10]:
combo.head()

,age,became_member_on,gender,person_id,income,event,person,time,value
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,transaction,0610b486422d4921ae7d2bf64640c50b,18,{'amount': 21.51}
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,transaction,0610b486422d4921ae7d2bf64640c50b,144,{'amount': 32.28}
2,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,0610b486422d4921ae7d2bf64640c50b,408,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
3,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,0610b486422d4921ae7d2bf64640c50b,504,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
4,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,transaction,0610b486422d4921ae7d2bf64640c50b,528,{'amount': 23.22}


In [11]:
combo['age'].value_counts()

58    7546
53    6688
51    6559
54    6546
57    6468
55    6382
59    6308
52    6246
63    6087
60    6013
56    5951
62    5696
67    5667
49    5656
64    5579
61    5533
48    5157
66    5133
50    5073
65    5047
68    4706
69    4465
70    4395
45    4276
47    4149
71    4052
43    4051
46    3995
42    3993
40    3942
      ... 
20    2582
77    2570
23    2563
35    2536
29    2468
22    2430
78    2427
80    2121
81    2071
31    2066
83    1991
79    1973
82    1884
84    1510
18    1367
85    1286
87    1213
88    1127
86     976
89     968
90     829
91     790
92     756
94     606
93     541
95     369
97     239
96     135
99      99
98      85
Name: age, Length: 82, dtype: int64

In [12]:
from pandas.io.json import json_normalize

In [13]:
offer_df = json_normalize(combo['value'])

In [14]:
offer_df.head()

,amount,offer id,offer_id,reward
0,21.51,NaN,NaN,NaN
1,32.28,NaN,NaN,NaN
2,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
3,NaN,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,23.22,NaN,NaN,NaN


In [15]:
offer_df['off_id'] = offer_df['offer id'].combine_first(offer_df['offer_id'])

In [16]:
offer_df.head(20)

,amount,offer id,offer_id,reward,off_id
0,21.51,NaN,NaN,NaN,NaN
1,32.28,NaN,NaN,NaN,NaN
2,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,NaN,3f207df678b143eea3cee63160fa8bed,NaN,NaN,3f207df678b143eea3cee63160fa8bed
4,23.22,NaN,NaN,NaN,NaN
5,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
6,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
7,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
8,19.89,NaN,NaN,NaN,NaN
9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,9b98b8c7a33c4b65b9aebfe6a799e6d9


In [17]:
combo_df = pd.concat([combo, offer_df], axis=1)

In [18]:
combo_df.head(2)

,age,became_member_on,gender,person_id,income,event,person,time,value,amount,offer id,offer_id,reward,off_id
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,transaction,0610b486422d4921ae7d2bf64640c50b,18,{'amount': 21.51},21.51,NaN,NaN,NaN,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,transaction,0610b486422d4921ae7d2bf64640c50b,144,{'amount': 32.28},32.28,NaN,NaN,NaN,NaN


In [19]:
df_offer_final = pd.merge(combo_df, portfolio, how='inner', left_on=['off_id'], right_on=['id']).rename(columns={'reward_y': 'reward'})

In [20]:
df_offer_final.head(3)

,age,became_member_on,gender,person_id,income,event,person,time,value,amount,offer id,offer_id,reward_x,off_id,channels,difficulty,duration,id,offer_type,reward
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,0610b486422d4921ae7d2bf64640c50b,408,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer completed,0610b486422d4921ae7d2bf64640c50b,528,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
2,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5


In [21]:
df_offer_final = df_offer_final[['age', 'became_member_on', 'gender',
                                 'person_id', 'income', 'event', 'time', 'off_id', 'channels', 
                                'difficulty', 'duration', 'offer_type', 'reward']]

In [22]:
df_offer_final.head(3)

,age,became_member_on,gender,person_id,income,event,time,off_id,channels,difficulty,duration,offer_type,reward
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer completed,528,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5
2,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5


In [23]:
df_offer_final.loc[(df_offer_final['age'] < 20), 'age_group'] = 'UNDER_20'
df_offer_final.loc[(df_offer_final['age'] >= 20) & (df_offer_final['age'] <= 40), 'age_group'] = '20_40'
df_offer_final.loc[(df_offer_final['age'] > 40) & (df_offer_final['age'] <= 60), 'age_group'] = '41_60'
df_offer_final.loc[(df_offer_final['age'] > 60) & (df_offer_final['age'] <= 80), 'age_group'] = '61_80'
df_offer_final.loc[(df_offer_final['age'] > 80), 'age_group'] = 'ABOVE_80'

In [24]:
df_offer_final.head(2)

,age,became_member_on,gender,person_id,income,event,time,off_id,channels,difficulty,duration,offer_type,reward,age_group
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5,41_60
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer completed,528,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5,41_60


In [25]:
from datetime import datetime
current_year_month = int(datetime.today().strftime("%Y%m"))
current_year_month

202109

In [26]:
df_offer_final['membership_age'] = current_year_month - df_offer_final['became_member_on'].apply(lambda x: int(str(x)[:6]))

In [27]:
# df_offer_final.loc[(df_offer_final.groupby(['person_id', 'off_id'])['age'].count()>=2) & 
#                   (df_offer_final['event']=='offer completed')] = 1

In [28]:
df_grouped = pd.DataFrame(
    df_offer_final.groupby(['person_id','off_id']).count()['age'])
df_grouped = df_grouped.rename(columns={"age":"count"})
df_grouped_agg = pd.DataFrame(df_offer_final.groupby(['person_id','off_id'])['event'].agg(list))
df_grouped_agg = df_grouped_agg.rename(columns={"event": "event_lst"})
df_grouped = pd.merge(df_grouped, df_grouped_agg, how='inner', left_on=['person_id', 'off_id'],
                    right_on=['person_id', 'off_id'])
df_grouped.head(2)

count  \
person_id                        off_id                                    
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5      2   
                                 3f207df678b143eea3cee63160fa8bed      2   

                                                                                           event_lst  
person_id                        off_id                                                               
0009655768c64bdeb2e877511632db8f 2906b810c7d4411798c6938adc9daaa5  [offer received, offer completed]  
                                 3f207df678b143eea3cee63160fa8bed     [offer received, offer viewed]

In [30]:
df_final = pd.merge(df_offer_final, df_grouped, how='inner', left_on=['person_id', 'off_id'],
                    right_on=['person_id', 'off_id'])

In [31]:
df_final.head(2)

,age,became_member_on,gender,person_id,income,event,time,off_id,channels,difficulty,duration,offer_type,reward,age_group,membership_age,count,event_lst
0,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer received,408,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5,41_60,402,2,"[offer received, offer completed]"
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0,offer completed,528,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,7,bogo,5,41_60,402,2,"[offer received, offer completed]"


In [38]:
df_final = df_final.drop('time', axis=1)

In [40]:
df_final.count()

age                 148628
became_member_on    148628
gender              148628
person_id           148628
income              148628
event               148628
off_id              148628
channels            148628
difficulty          148628
duration            148628
offer_type          148628
reward              148628
age_group           148628
membership_age      148628
count               148628
event_lst           148628
dtype: int64

In [46]:
for df_final['event_lst'].iterrows()

0                         [offer received, offer completed]
1                         [offer received, offer completed]
2           [offer received, offer viewed, offer completed]
3           [offer received, offer viewed, offer completed]
4           [offer received, offer viewed, offer completed]
5           [offer received, offer viewed, offer completed]
6           [offer received, offer viewed, offer completed]
7           [offer received, offer viewed, offer completed]
8         [offer received, offer viewed, offer received,...
9         [offer received, offer viewed, offer received,...
10        [offer received, offer viewed, offer received,...
11        [offer received, offer viewed, offer received,...
12        [offer received, offer viewed, offer received,...
13        [offer received, offer received, offer completed]
14        [offer received, offer received, offer completed]
15        [offer received, offer received, offer completed]
16                                      

In [36]:
df_offer_final['event'].value_counts()

offer received     66427
offer viewed       49802
offer completed    32399
Name: event, dtype: int64

In [101]:
df_offer_final.groupby(['person_id','off_id'])['event'].agg(list)

['offer received',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer completed',
 'offer viewed',
 'offer received',
 'offer completed',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer completed',
 'offer received',
 'offer completed',
 'offer received',
 'offer viewed',
 'offer received',
 'offer viewed',
 'offer received

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
plt.plot(df_offer_final['gender'], df_offer_final['event'])
plt.show()

In [ ]:
X = df_offer_final[['age', 'gender', 'income', 'time']]